In [4]:
import os
import sys
sys.path.append('D:\实习\研报复现\QuantDev\ymtool')
import numpy as np
import pandas as pd
from tqdm import tqdm
import statsmodels.api as sm

import Alpha as alpha

In [5]:
close = alpha.load_data('pv\\stock_close')
low = alpha.load_data('pv\\stock_low')
high = alpha.load_data('pv\\stock_high')

PLUS = (2*close-high-low).div(close.shift(1),axis=0).dropna(axis=0,how='all')

turnover = alpha.load_data('pv\\stock_turnover').replace(0,np.nan)

In [10]:
def nonneg(fct):
    min_val = fct.min(axis=1)
    nonneg_fct = fct.sub(min_val,axis=0)
    return nonneg_fct

def neutral(fct1,fct2):
    res = []
    for idx in tqdm(fct1.index):
        temp = pd.concat([fct1.loc[idx],fct2.loc[idx]],axis=1).dropna()

        X = temp.iloc[:,1:]
        X = sm.add_constant(X)
        y = temp.iloc[:,0]

        # 创建模型
        model = sm.OLS(y, X)

        # 拟合模型
        try:
            results = model.fit()
        except:
            print(idx)
            break
        res.append(results.resid)

    fctNeu = alpha.zscore(pd.concat(res,axis=1).T)
    fctNeu.index = fct1.index
    return fctNeu

In [11]:
Turn_dePLUS = neutral(turnover.iloc[1:],PLUS.replace([np.inf,-np.inf],np.nan))
Turn20_dePLUS = Turn_dePLUS.rolling(20,min_periods=1).mean().dropna(axis=0,how='all').iloc[19:]
STR_dePLUS =Turn_dePLUS.rolling(20,min_periods=1).std().dropna(axis=0,how='all').iloc[19:]

PLUS_deTurn = neutral(PLUS,turnover)
PLUS_deTurn20 = PLUS_deTurn.rolling(20,min_periods=1).mean().dropna(axis=0,how='all').iloc[19:]


TPS = nonneg(alpha.zscore(Turn20_dePLUS)).mul(nonneg(alpha.zscore(PLUS_deTurn20)),axis=0)

SPS = nonneg(alpha.zscore(STR_dePLUS)).mul(nonneg(alpha.zscore(PLUS_deTurn20)),axis=0)

100%|██████████████████████████████████████████████████████████████████████████████| 3702/3702 [00:39<00:00, 92.77it/s]


In [13]:
TPSNeu = alpha.mvind_neutral(TPS.dropna(axis=0,how='all'))

100%|██████████████████████████████████████████████████████████████████████████████| 3683/3683 [00:58<00:00, 63.33it/s]


In [14]:
SPSNeu = alpha.mvind_neutral(SPS.dropna(axis=0,how='all'))

100%|██████████████████████████████████████████████████████████████████████████████| 3682/3682 [00:51<00:00, 72.10it/s]


In [15]:
alpha.to_fctDataBase(TPSNeu,'TPSNeu')
fct = alpha.load_fct('TPSNeu')
alpha.Alpha(fct, 'TPSNeu', verbose=True, freq='W', layer=10, rolling_ic=True).main()

开始计算Barra相关性
开始计算RankIC
清除ST
清除涨跌停
获取交易面板完成
开始分层回测...
开始计算分层超额...
结果已经存入D:\实习\研报复现\QuantResearch\选股因子回测结果\report\TPSNeu_W.xlsx


In [16]:
alpha.to_fctDataBase(SPSNeu,'SPSNeu')
fct = alpha.load_fct('SPSNeu')
alpha.Alpha(fct, 'SPSNeu', verbose=True, freq='W', layer=10, rolling_ic=True).main()

开始计算Barra相关性
开始计算RankIC
清除ST
清除涨跌停
获取交易面板完成
开始分层回测...
开始计算分层超额...
结果已经存入D:\实习\研报复现\QuantResearch\选股因子回测结果\report\SPSNeu_W.xlsx
